https://blog.paperspace.com/implementing-gans-in-tensorflow/

In [13]:
import numpy as np

def get_y(x):
    return 10 + x*x


def sample_data(n=10000, scale=100):
    data = []
    
    x = scale*(np.random.random_sample((n,))-0.5)
    
    for i in range(n):
        yi = get_y(x[i])
        data.append([x[i],yi])
        
    return np.array(data)

data = sample_data

In [14]:
import matplotlib.pyplot as plt

xy = sample_data()

x = xy[:,:1].reshape(10000)
y = xy[:,1:].reshape(10000)


xy.reshape((10000,2))

array([[  26.03837185,  687.99680874],
       [  -4.96007032,   34.60229757],
       [  16.94860265,  297.25513175],
       ...,
       [ -41.27118767, 1713.31093154],
       [  12.30482919,  161.40882139],
       [  41.25469896, 1711.95018616]])

In [15]:
import tensorflow as tf

def generator(z, hsize=[16,16], reuse=False):
    with tf.variable_scope("GAN/Generator", reuse=reuse):
        h1 = tf.layers.dense(Z, hsize[0], activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1, hsize[1],activation = tf.nn.leaky_relu)
        out = tf.layers.dense(h2,2)
        
    return out

In [16]:
def discriminator(X, hsize=[16,16], reuse=False):
    with tf.variable_scope("GAN/Discriminator", reuse=reuse):
        h1 = tf.layers.dense(X, hsize[0], activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1, hsize[1], activation=tf.nn.leaky_relu)
        h3 = tf.layers.dense(h2,2)
        out = tf.layers.dense(h3,1)
        
    return out,h3

In [17]:
X = tf.placeholder(tf.float32, [None,2])
Z = tf.placeholder(tf.float32, [None,2])

In [18]:
G_sample = generator(Z)
r_logits, r_rep = discriminator(X)
f_logits, g_rep = discriminator(G_sample, reuse=True)

ValueError: Variable GAN/Generator/dense/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-3-1a85566c1676>", line 5, in generator
    h1 = tf.layers.dense(Z, hsize[0], activation=tf.nn.leaky_relu)
  File "<ipython-input-6-7373e485cfe4>", line 1, in <module>
    G_sample = generator(Z)
  File "/home/user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
disc_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits,labels=tf.ones_like(r_logits)) + tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.zeros_like(f_logits)))
gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.ones_like(f_logits)))

In [20]:
gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Generator")
disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Discriminator")

gen_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(gen_loss,var_list = gen_vars) # G Train step
disc_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(disc_loss,var_list = disc_vars) # D Train step



ValueError: Variable GAN/Generator/dense/kernel/RMSProp/ already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-8-8c971c394689>", line 4, in <module>
    gen_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(gen_loss,var_list = gen_vars) # G Train step
  File "/home/user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2903, in run_ast_nodes
    if self.run_code(code, result):


In [21]:
batch_size=1000

for i in range(100001):
    X_batch = sample_data(n=batch_size)
    Z_batch = sample_Z(batch_size, 2)
    _, dloss = sess.run([disc_step, disc_loss], feed_dict={X: X_batch, Z: Z_batch})
    _, gloss = sess.run([gen_step, gen_loss], feed_dict={Z: Z_batch})

    print("Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(i,dloss,gloss))


NameError: name 'sample_Z' is not defined

In [1]:
import numpy as np
import math

def get_y(x):
    return 10 + x*x


def sample_data(n=10000, scale=100):
    data = []

    x = scale*(np.random.random_sample((n,))-0.5)

    for i in range(n):
        yi = get_y(x[i])
        data.append([x[i], yi])

    return np.array(data)

In [2]:
import tensorflow as tf
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
sb.set()

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def generator(Z,hsize=[16, 16],reuse=False):
    with tf.variable_scope("GAN/Generator",reuse=reuse):
        h1 = tf.layers.dense(Z,hsize[0],activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
        out = tf.layers.dense(h2,2)

    return out

def discriminator(X,hsize=[16, 16],reuse=False):
    with tf.variable_scope("GAN/Discriminator",reuse=reuse):
        h1 = tf.layers.dense(X,hsize[0],activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
        h3 = tf.layers.dense(h2,2)
        out = tf.layers.dense(h3,1)

    return out, h3


X = tf.placeholder(tf.float32,[None,2])
Z = tf.placeholder(tf.float32,[None,2])

G_sample = generator(Z)
r_logits, r_rep = discriminator(X)
f_logits, g_rep = discriminator(G_sample,reuse=True)

disc_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits,labels=tf.ones_like(r_logits)) + tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.zeros_like(f_logits)))
gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.ones_like(f_logits)))

gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Generator")
disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Discriminator")

gen_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(gen_loss,var_list = gen_vars) # G Train step
disc_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(disc_loss,var_list = disc_vars) # D Train step



# sess = tf.Session(config=config)
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)

batch_size = 256
nd_steps = 10
ng_steps = 10

x_plot = sample_data(n=batch_size)

f = open('loss_logs.csv','w')
f.write('Iteration,Discriminator Loss,Generator Loss\n')

for i in range(10001):
    X_batch = sample_data(n=batch_size)
    Z_batch = sample_Z(batch_size, 2)

    for _ in range(nd_steps):
        _, dloss = sess.run([disc_step, disc_loss], feed_dict={X: X_batch, Z: Z_batch})
    rrep_dstep, grep_dstep = sess.run([r_rep, g_rep], feed_dict={X: X_batch, Z: Z_batch})

    for _ in range(ng_steps):
        _, gloss = sess.run([gen_step, gen_loss], feed_dict={Z: Z_batch})

    rrep_gstep, grep_gstep = sess.run([r_rep, g_rep], feed_dict={X: X_batch, Z: Z_batch})

    print("Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(i,dloss,gloss))
    if i%10 == 0:
        f.write("%d,%f,%f\n"%(i,dloss,gloss))

    if i%1000 == 0:
        plt.figure()
        g_plot = sess.run(G_sample, feed_dict={Z: Z_batch})
        xax = plt.scatter(x_plot[:,0], x_plot[:,1])
        gax = plt.scatter(g_plot[:,0],g_plot[:,1])

        plt.legend((xax,gax), ("Real Data","Generated Data"))
        plt.title('Samples at Iteration %d'%i)
        plt.tight_layout()
        #plt.savefig('../plots/iterations/iteration_%d.png'%i)
        plt.close()

        plt.figure()
        rrd = plt.scatter(rrep_dstep[:,0], rrep_dstep[:,1], alpha=0.5)
        rrg = plt.scatter(rrep_gstep[:,0], rrep_gstep[:,1], alpha=0.5)
        grd = plt.scatter(grep_dstep[:,0], grep_dstep[:,1], alpha=0.5)
        grg = plt.scatter(grep_gstep[:,0], grep_gstep[:,1], alpha=0.5)


        plt.legend((rrd, rrg, grd, grg), ("Real Data Before G step","Real Data After G step",
                               "Generated Data Before G step","Generated Data After G step"))
        plt.title('Transformed Features at Iteration %d'%i)
        plt.tight_layout()
        #plt.savefig('../plots/features/feature_transform_%d.png'%i)
        plt.close()

        plt.figure()

        rrdc = plt.scatter(np.mean(rrep_dstep[:,0]), np.mean(rrep_dstep[:,1]),s=100, alpha=0.5)
        rrgc = plt.scatter(np.mean(rrep_gstep[:,0]), np.mean(rrep_gstep[:,1]),s=100, alpha=0.5)
        grdc = plt.scatter(np.mean(grep_dstep[:,0]), np.mean(grep_dstep[:,1]),s=100, alpha=0.5)
        grgc = plt.scatter(np.mean(grep_gstep[:,0]), np.mean(grep_gstep[:,1]),s=100, alpha=0.5)

        plt.legend((rrdc, rrgc, grdc, grgc), ("Real Data Before G step","Real Data After G step",
                               "Generated Data Before G step","Generated Data After G step"))

        plt.title('Centroid of Transformed Features at Iteration %d'%i)
        plt.tight_layout()
        #plt.savefig('../plots/features/feature_transform_centroid_%d.png'%i)
        plt.close()

f.close()


/home/user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Iterations: 0	 Discriminator loss: 0.7045	 Generator loss: 0.6861
Iterations: 1	 Discriminator loss: 0.6980	 Generator loss: 0.6930
Iterations: 2	 Discriminator loss: 0.6577	 Generator loss: 0.7337
Iterations: 3	 Discriminator loss: 0.6300	 Generator loss: 0.7623
Iterations: 4	 Discriminator loss: 0.5877	 Generator loss: 0.8111
Iterations: 5	 Discriminator loss: 0.5376	 Generator loss: 0.8739
Iterations: 6	 Discriminator loss: 0.4947	 Generator loss: 0.9336
Iterations: 7	 Discriminator loss: 0.4383	 Generator loss: 1.0301
Iterations: 8	 Discriminator loss: 0.3962	 Generator loss: 1.1083
Iterations: 9	 Discriminator loss: 0.3436	 Generator loss: 1.2333
Iterations: 10	 Discriminator loss: 0.2817	 Generator loss: 1.4099
Iterations: 11	 Discriminator loss: 0.2314	 Generator loss: 1.5769
Iterations: 12	 Discriminator loss: 0.1864	 Generator loss: 1.7582
Iterations: 13	 Discriminator loss: 0.1530	 Generator loss: 1.9179
Iterations: 14	 Discriminator loss: 0.1269	 Generator loss: 2.0938
Itera

Iterations: 126	 Discriminator loss: 0.9713	 Generator loss: 1.8685
Iterations: 127	 Discriminator loss: 3.7347	 Generator loss: 1.3561
Iterations: 128	 Discriminator loss: 3.2191	 Generator loss: 0.0066
Iterations: 129	 Discriminator loss: 0.9976	 Generator loss: 2.2008
Iterations: 130	 Discriminator loss: 1.1300	 Generator loss: 2.0464
Iterations: 131	 Discriminator loss: 1.1404	 Generator loss: 2.6488
Iterations: 132	 Discriminator loss: 1.5633	 Generator loss: 0.2454
Iterations: 133	 Discriminator loss: 0.9801	 Generator loss: 2.1592
Iterations: 134	 Discriminator loss: 2.2867	 Generator loss: 5.0962
Iterations: 135	 Discriminator loss: 1.2690	 Generator loss: 0.6210
Iterations: 136	 Discriminator loss: 1.0723	 Generator loss: 2.7539
Iterations: 137	 Discriminator loss: 12.0388	 Generator loss: 0.0301
Iterations: 138	 Discriminator loss: 3.0697	 Generator loss: 4.6580
Iterations: 139	 Discriminator loss: 0.9695	 Generator loss: 1.9692
Iterations: 140	 Discriminator loss: 1.1781	 Ge

Iterations: 248	 Discriminator loss: 12.9601	 Generator loss: 0.0513
Iterations: 249	 Discriminator loss: 6.3111	 Generator loss: 0.0002
Iterations: 250	 Discriminator loss: 13.0472	 Generator loss: 0.0573
Iterations: 251	 Discriminator loss: 6.6104	 Generator loss: 0.0001
Iterations: 252	 Discriminator loss: 10.8144	 Generator loss: 0.0131
Iterations: 253	 Discriminator loss: 13.8445	 Generator loss: 0.1249
Iterations: 254	 Discriminator loss: 1.9645	 Generator loss: 0.1386
Iterations: 255	 Discriminator loss: 3.7852	 Generator loss: 24.6976
Iterations: 256	 Discriminator loss: 1.9201	 Generator loss: 0.1761
Iterations: 257	 Discriminator loss: 12.2801	 Generator loss: 0.0501
Iterations: 258	 Discriminator loss: 15.1687	 Generator loss: 0.6587
Iterations: 259	 Discriminator loss: 7.5754	 Generator loss: 0.0001
Iterations: 260	 Discriminator loss: 13.0819	 Generator loss: 0.1565
Iterations: 261	 Discriminator loss: 8.7304	 Generator loss: 0.0157
Iterations: 262	 Discriminator loss: 16.

Iterations: 372	 Discriminator loss: 5.1968	 Generator loss: 9.1435
Iterations: 373	 Discriminator loss: 3.1976	 Generator loss: 10.6466
Iterations: 374	 Discriminator loss: 2.0553	 Generator loss: 7.2834
Iterations: 375	 Discriminator loss: 6.7509	 Generator loss: 8.0371
Iterations: 376	 Discriminator loss: 1.4694	 Generator loss: 5.2219
Iterations: 377	 Discriminator loss: 3.4362	 Generator loss: 12.6392
Iterations: 378	 Discriminator loss: 3.6922	 Generator loss: 11.0827
Iterations: 379	 Discriminator loss: 6.0119	 Generator loss: 8.4233
Iterations: 380	 Discriminator loss: 5.9417	 Generator loss: 8.2114
Iterations: 381	 Discriminator loss: 4.8978	 Generator loss: 9.1769
Iterations: 382	 Discriminator loss: 4.3685	 Generator loss: 9.5442
Iterations: 383	 Discriminator loss: 6.2651	 Generator loss: 7.2784
Iterations: 384	 Discriminator loss: 5.9665	 Generator loss: 8.2170
Iterations: 385	 Discriminator loss: 6.7877	 Generator loss: 7.3156
Iterations: 386	 Discriminator loss: 6.7528	 

...

Iterations: 2315	 Discriminator loss: 1.0395	 Generator loss: 0.6881
Iterations: 2316	 Discriminator loss: 1.0667	 Generator loss: 0.8293
Iterations: 2317	 Discriminator loss: 1.0845	 Generator loss: 0.6276
Iterations: 2318	 Discriminator loss: 1.0737	 Generator loss: 0.8814
Iterations: 2319	 Discriminator loss: 1.0900	 Generator loss: 0.6687
Iterations: 2320	 Discriminator loss: 1.0760	 Generator loss: 0.9124
Iterations: 2321	 Discriminator loss: 1.1029	 Generator loss: 0.7159
Iterations: 2322	 Discriminator loss: 1.1375	 Generator loss: 0.7717
Iterations: 2323	 Discriminator loss: 1.1384	 Generator loss: 0.7740
Iterations: 2324	 Discriminator loss: 1.1046	 Generator loss: 0.7272
Iterations: 2325	 Discriminator loss: 0.9921	 Generator loss: 0.8864
Iterations: 2326	 Discriminator loss: 1.0520	 Generator loss: 0.5134
Iterations: 2327	 Discriminator loss: 1.0987	 Generator loss: 0.8863
Iterations: 2328	 Discriminator loss: 1.1087	 Generator loss: 0.7772
Iterations: 2329	 Discriminator lo

Iterations: 2438	 Discriminator loss: 1.0601	 Generator loss: 0.7486
Iterations: 2439	 Discriminator loss: 1.0641	 Generator loss: 0.9775
Iterations: 2440	 Discriminator loss: 1.1112	 Generator loss: 0.8435
Iterations: 2441	 Discriminator loss: 0.9731	 Generator loss: 0.7828
Iterations: 2442	 Discriminator loss: 1.0855	 Generator loss: 1.2016
Iterations: 2443	 Discriminator loss: 0.9864	 Generator loss: 0.9594
Iterations: 2444	 Discriminator loss: 1.0641	 Generator loss: 0.6501
Iterations: 2445	 Discriminator loss: 1.0637	 Generator loss: 0.8969
Iterations: 2446	 Discriminator loss: 1.0348	 Generator loss: 0.6684
Iterations: 2447	 Discriminator loss: 1.0704	 Generator loss: 0.8752
Iterations: 2448	 Discriminator loss: 1.0989	 Generator loss: 0.7274
Iterations: 2449	 Discriminator loss: 1.0750	 Generator loss: 0.9020
Iterations: 2450	 Discriminator loss: 1.0896	 Generator loss: 0.7444
Iterations: 2451	 Discriminator loss: 1.0888	 Generator loss: 0.7125
Iterations: 2452	 Discriminator lo

Iterations: 2559	 Discriminator loss: 1.0569	 Generator loss: 0.8399
Iterations: 2560	 Discriminator loss: 1.1182	 Generator loss: 0.6590
Iterations: 2561	 Discriminator loss: 1.0808	 Generator loss: 0.8642
Iterations: 2562	 Discriminator loss: 1.0943	 Generator loss: 0.6852
Iterations: 2563	 Discriminator loss: 1.0496	 Generator loss: 0.8607
Iterations: 2564	 Discriminator loss: 1.1019	 Generator loss: 0.6006
Iterations: 2565	 Discriminator loss: 1.0606	 Generator loss: 0.8821
Iterations: 2566	 Discriminator loss: 1.0985	 Generator loss: 0.7014
Iterations: 2567	 Discriminator loss: 1.0266	 Generator loss: 0.9345
Iterations: 2568	 Discriminator loss: 1.0943	 Generator loss: 0.6373
Iterations: 2569	 Discriminator loss: 0.9752	 Generator loss: 0.8605
Iterations: 2570	 Discriminator loss: 1.0474	 Generator loss: 0.6275
Iterations: 2571	 Discriminator loss: 1.1030	 Generator loss: 0.8778
Iterations: 2572	 Discriminator loss: 1.0593	 Generator loss: 0.6441
Iterations: 2573	 Discriminator lo

Iterations: 2678	 Discriminator loss: 1.0931	 Generator loss: 0.8398
Iterations: 2679	 Discriminator loss: 1.0759	 Generator loss: 0.7393
Iterations: 2680	 Discriminator loss: 1.1120	 Generator loss: 0.7243
Iterations: 2681	 Discriminator loss: 1.0971	 Generator loss: 0.5458
Iterations: 2682	 Discriminator loss: 1.0764	 Generator loss: 0.8707
Iterations: 2683	 Discriminator loss: 1.1324	 Generator loss: 0.7037
Iterations: 2684	 Discriminator loss: 1.1342	 Generator loss: 0.7786
Iterations: 2685	 Discriminator loss: 1.0996	 Generator loss: 0.8382
Iterations: 2686	 Discriminator loss: 1.1439	 Generator loss: 0.6717
Iterations: 2687	 Discriminator loss: 1.0854	 Generator loss: 0.8688
Iterations: 2688	 Discriminator loss: 1.0876	 Generator loss: 0.5705
Iterations: 2689	 Discriminator loss: 1.0605	 Generator loss: 0.8265
Iterations: 2690	 Discriminator loss: 1.1650	 Generator loss: 0.7141
Iterations: 2691	 Discriminator loss: 1.0997	 Generator loss: 0.9085
Iterations: 2692	 Discriminator lo

KeyboardInterrupt: 